In [1]:
# -*- coding: utf-8 -*-
import os
import re
import time
import unicodedata
import datetime as dt

import pandas as pd
import requests
from bs4 import BeautifulSoup
from requests.adapters import HTTPAdapter, Retry

# -----------------------------
# Config
# -----------------------------
TICKERS = [
    "ABJC","BICB","BICC","BNBC","BOAB","BOABF","BOAC","BOAM","BOAN","BOAS",
    "CABC","CBIBF","CFAC","CIEC","ECOC","ETIT","FTSC","LNBB","NEIC","NSBC",
    "NTLC","ONTBF","ORAC","ORGT","PALC","PRSC","SAFC","SCRC","SDCC","SDSC",
    "SEMC","SGBC","SHEC","SIBC","SICC","SIVC","SLBC","SMBC","SNTS","SOGC",
    "SPHC","STAC","STBC","SVOC","TTLC","TTLS","UNLC","UNXC"
]

BASE_URL = "https://www.richbourse.com/common/variation/historique/{ticker}/jour/0/{end}?page={page}"
HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 14_6) "
        "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0 Safari/537.36"
    )
}

TARGET_COLS = [
    "Ticker",
    "Date",
    "Variation",
    "Volume Devise Total",
    "Cours Ajuste",
    "Volume Ajuste Total",
    "Cours Normal",
    "Volume Normal Total",
]

CSV_PATH = "combined_journalier.csv"
MAX_PAGES = 60
REQUEST_PAUSE = 0.30

# -----------------------------
# Utils
# -----------------------------
def _strip_all(s: str) -> str:
    s = str(s).replace("\ufeff", "").replace("\xa0", " ")
    s = unicodedata.normalize("NFKC", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def parse_date_any(series: pd.Series) -> pd.Series:
    """Parse dates possibles (ISO, dd/mm/YYYY, mm/dd/YYYY)."""
    s = series.astype(str).str.strip()
    d1 = pd.to_datetime(s, format="%Y-%m-%d", errors="coerce")
    d2 = pd.to_datetime(s, format="%d/%m/%Y", errors="coerce")
    d3 = pd.to_datetime(s, format="%m/%d/%Y", errors="coerce")
    return d1.fillna(d2).fillna(d3)

def read_csv_flex(path: str) -> pd.DataFrame:
    df = pd.read_csv(
        path,
        sep=None,
        engine="python",
        encoding="utf-8-sig",
        dtype=str,
        on_bad_lines="skip",
        quoting=3,
        escapechar="\\",
        keep_default_na=False,
    )
    return df

def save_csv_atomic(df: pd.DataFrame, path: str):
    tmp = f"{path}.tmp"
    df.to_csv(tmp, index=False)
    os.replace(tmp, path)

def make_session() -> requests.Session:
    s = requests.Session()
    retries = Retry(
        total=3,
        backoff_factor=0.6,
        status_forcelist=[429, 500, 502, 503, 504],
        allowed_methods=["GET"],
        raise_on_status=False,
    )
    s.headers.update(HEADERS)
    s.mount("https://", HTTPAdapter(max_retries=retries))
    s.mount("http://", HTTPAdapter(max_retries=retries))
    return s

def clean_num(x: str) -> str:
    return x.replace("\xa0", " ").replace(" ", "").replace(",", ".")

# -----------------------------
# Scraping avec pagination & cutoff
# -----------------------------
def scrape_ticker_since(ticker: str, last_date: dt.date, session: requests.Session) -> pd.DataFrame:
    end_str = dt.date.today().strftime("%d-%m-%Y")
    collected = []

    for page in range(1, MAX_PAGES + 1):
        url = BASE_URL.format(ticker=ticker, end=end_str, page=page)
        r = session.get(url, timeout=20)
        r.raise_for_status()
        soup = BeautifulSoup(r.text, "html.parser")

        table = soup.find("table", {"class": "table"})
        if table is None:
            break
        tbody = table.find("tbody")
        if tbody is None:
            break

        stop = False
        for tr in tbody.find_all("tr"):
            tds = [td.get_text(strip=True) for td in tr.find_all("td")]
            if len(tds) < 7:
                continue
            try:
                d = dt.datetime.strptime(tds[0], "%d/%m/%Y").date()
            except ValueError:
                continue

            if last_date is not None and d <= last_date:
                stop = True
                break

            collected.append({
                "Ticker": f"{ticker} J",
                "Date": d.strftime("%d/%m/%Y"),
                "Variation": clean_num(tds[1]).replace("%", ""),
                "Volume Devise Total": clean_num(tds[2]),
                "Cours Ajuste": clean_num(tds[3]),
                "Volume Ajuste Total": clean_num(tds[4]),
                "Cours Normal": clean_num(tds[5]),
                "Volume Normal Total": clean_num(tds[6]),
                "Date_dt": pd.Timestamp(d),
            })

        if stop:
            break
        time.sleep(REQUEST_PAUSE)

    if not collected:
        return pd.DataFrame(columns=TARGET_COLS + ["Date_dt"])
    return pd.DataFrame(collected)

# -----------------------------
# Update principal
# -----------------------------
def update_csv(csv_path: str = CSV_PATH):
    if os.path.exists(csv_path):
        base = read_csv_flex(csv_path)
        for col in TARGET_COLS:
            if col not in base.columns:
                base[col] = pd.NA
        base = base[TARGET_COLS]
    else:
        base = pd.DataFrame(columns=TARGET_COLS)

    if not base.empty:
        base["Ticker"] = (
            base["Ticker"].astype(str)
            .str.replace(r"\s+J$", "", regex=True)
            .str.strip() + " J"
        )
        base["Date"] = base["Date"].astype(str).map(_strip_all)

    base["Date_dt"] = parse_date_any(base["Date"])

    session = make_session()
    updated_any = False

    for ticker in TICKERS:
        mask_t = base["Ticker"] == f"{ticker} J"
        last_date = None
        if mask_t.any():
            max_ts = base.loc[mask_t, "Date_dt"].max()
            if pd.notna(max_ts):
                last_date = max_ts.date()

        df_new = scrape_ticker_since(ticker, last_date, session)

        if not df_new.empty:
            updated_any = True
            base = pd.concat([base, df_new], ignore_index=True)
            base = base.drop_duplicates(subset=["Ticker", "Date_dt"])
            print(f"{len(df_new)} lignes ajoutées pour {ticker}")
        else:
            print(f"Aucune nouvelle ligne pour {ticker}")

        time.sleep(REQUEST_PAUSE)

    # TRI final : par ticker puis date décroissante
    base = base.sort_values(["Ticker", "Date_dt"], ascending=[True, False], na_position="last")
    base["Date"] = base["Date_dt"].dt.strftime("%d/%m/%Y")
    base = base[TARGET_COLS]

    save_csv_atomic(base, csv_path)
    if updated_any:
        print("Table mise à jour :", csv_path)
    else:
        print("table à jour (tri/format réappliqués)")

# -----------------------------
# Entrypoint
# -----------------------------
if __name__ == "__main__":
    update_csv(CSV_PATH)


3 lignes ajoutées pour ABJC
3 lignes ajoutées pour BICB
3 lignes ajoutées pour BICC
3 lignes ajoutées pour BNBC
3 lignes ajoutées pour BOAB
3 lignes ajoutées pour BOABF
3 lignes ajoutées pour BOAC
3 lignes ajoutées pour BOAM
3 lignes ajoutées pour BOAN
3 lignes ajoutées pour BOAS
3 lignes ajoutées pour CABC
3 lignes ajoutées pour CBIBF
3 lignes ajoutées pour CFAC
3 lignes ajoutées pour CIEC
3 lignes ajoutées pour ECOC
3 lignes ajoutées pour ETIT
3 lignes ajoutées pour FTSC
3 lignes ajoutées pour LNBB
3 lignes ajoutées pour NEIC
3 lignes ajoutées pour NSBC
3 lignes ajoutées pour NTLC
3 lignes ajoutées pour ONTBF
3 lignes ajoutées pour ORAC
3 lignes ajoutées pour ORGT
3 lignes ajoutées pour PALC
3 lignes ajoutées pour PRSC
3 lignes ajoutées pour SAFC
3 lignes ajoutées pour SCRC
3 lignes ajoutées pour SDCC
3 lignes ajoutées pour SDSC
Aucune nouvelle ligne pour SEMC
3 lignes ajoutées pour SGBC
3 lignes ajoutées pour SHEC
3 lignes ajoutées pour SIBC
3 lignes ajoutées pour SICC
3 lignes ajou